In [35]:
import warnings
from skimage.measure import compare_ssim
from skimage.transform import resize
from scipy.stats import wasserstein_distance
from scipy.misc import imsave
from scipy.ndimage import imread
import numpy as np
import cv2

##
# Globals
##

warnings.filterwarnings('ignore')

# specify resized image sizes
height = 2**10
width = 2**10

##
# Functions
##

def get_img(path, norm_size=True, norm_exposure=False):
    '''
    Prepare an image for image processing tasks
    '''
    # flatten returns a 2d grayscale array
    img = imread(path, flatten=True).astype(int)
    # resizing returns float vals 0:255; convert to ints for downstream tasks
    if norm_size:
        img = resize(img, (height, width), anti_aliasing=True, preserve_range=True)
    if norm_exposure:
        img = normalize_exposure(img)
    return img


def get_histogram(img):
    '''
    Get the histogram of an image. For an 8-bit, grayscale image, the
    histogram will be a 256 unit vector in which the nth value indicates
    the percent of the pixels in the image with the given darkness level.
    The histogram's values sum to 1.
    '''
    h, w = img.shape
    hist = [0.0] * 256
    for i in range(h):
        for j in range(w):
            hist[img[i, j]] += 1
    return np.array(hist) / (h * w) 


def normalize_exposure(img):
    '''
    Normalize the exposure of an image.
    '''
    img = img.astype(int)
    hist = get_histogram(img)
    # get the sum of vals accumulated by each position in hist
    cdf = np.array([sum(hist[:i+1]) for i in range(len(hist))])
    # determine the normalization values for each unit of the cdf
    sk = np.uint8(255 * cdf)
    # normalize each position in the output image
    height, width = img.shape
    normalized = np.zeros_like(img)
    for i in range(0, height):
        for j in range(0, width):
            normalized[i, j] = sk[img[i, j]]
    return normalized.astype(int)


def earth_movers_distance(path_a, path_b):
    '''
    Measure the Earth Mover's distance between two images
    @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
    @returns:
    TODO
    '''
    img_a = get_img(path_a, norm_exposure=True)
    img_b = get_img(path_b, norm_exposure=True)
    hist_a = get_histogram(img_a)
    hist_b = get_histogram(img_b)
    return wasserstein_distance(hist_a, hist_b)


def structural_sim(path_a, path_b):
    '''
    Measure the structural similarity between two images
    @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
    @returns:
    {float} a float {-1:1} that measures structural similarity
      between the input images
    '''
    img_a = get_img(path_a)
    img_b = get_img(path_b)
    sim, diff = compare_ssim(img_a, img_b, full=True)
    return sim


def pixel_sim(path_a, path_b):
    '''
    Measure the pixel-level similarity between two images
    @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
    @returns:
    {float} a float {-1:1} that measures structural similarity
      between the input images
    '''
    img_a = get_img(path_a, norm_exposure=True)
    img_b = get_img(path_b, norm_exposure=True)
    return np.sum(np.absolute(img_a - img_b)) / (height*width) / 255


def sift_sim(path_a, path_b):
    '''
    Use SIFT features to measure image similarity
    @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
    @returns:
    TODO
    '''
    # initialize the sift feature detector
#     orb = cv2.ORB_create()

#     # get the images
#     img_a = cv2.imread(path_a)
#     img_b = cv2.imread(path_b)

#     # find the keypoints and descriptors with SIFT
#     kp_a, desc_a = orb.detectAndCompute(img_a, None)
#     kp_b, desc_b = orb.detectAndCompute(img_b, None)

    desc_a = convert_sift(path_a)
#     print(desc_a.shape)
    desc_b = convert_sift(path_b)

    # initialize the bruteforce matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # match.distance is a float between {0:100} - lower means more similar
    matches = bf.match(desc_a, desc_b)
    similar_regions = [i for i in matches if i.distance < 70]
    if len(matches) == 0:
        return 0
    return len(similar_regions) / len(matches)

def convert_sift(path):
    # initialize the sift feature detector
    orb = cv2.ORB_create()

    # get the images
    img = cv2.imread(path)

    # find the keypoints and descriptors with SIFT
    kp, desc = orb.detectAndCompute(img, None)
    return desc



       
if __name__ == '__main__':
    img_a = '/home/topica/Desktop/anh_test4.jpg'
    img_b = '/home/topica/Desktop/anh_test1.png'
    # get the similarity values
    structural_sim = structural_sim(img_a, img_b)
    pixel_sim = pixel_sim(img_a, img_b)
    sift_sim = sift_sim(img_a, img_b)
    emd = earth_movers_distance(img_a, img_b)
    print(structural_sim, pixel_sim, sift_sim, emd)

0.4373420434779372 0.3701709148930568 0.9236111111111112 0.000715792179107666


In [ ]:
b56 0.4067628118300887 0.36775657803404566 1.0 0.00189102441072464
test1 0.4067628118300887 0.36775657803404566 1.0 0.00189102441072464
bai 0.35435528116248466 0.3931248010373583 1.0 0.002943076193332672

In [ ]:
427-1b 0.2009683167250975 0.08718918445063573 0.9735294117647059 0.0003592967987060547
bai_fa 0.05439038742425484 0.4403153587790096 0.7104072398190046 0.0047726258635520935
chu1.j 0.03087476570337006 0.29622936622769225 0.6201923076923077 0.0041794925928115845
demo_ro 0.03477050764628357 0.24145067252364813 0.9850746268656716 0.0010894611477851868
result_demo_rota 0.043052707973911 0.4053676867017559 0.7593360995850622 0.004580199718475342
result_hoa.jpg 0.0739527207578048 0.47579478095559513 0.743455497382199 0.005299217998981476

In [ ]:
#!/usr/bin/python3
# 2018.01.16 01:11:49 CST
# 2018.01.16 01:55:01 CST
import cv2
import numpy as np

## (1) read
img = cv2.imread("chu1.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## (2) threshold
th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)

## (3) minAreaRect on the nozeros
pts = cv2.findNonZero(threshed)
ret = cv2.minAreaRect(pts)

(cx,cy), (w,h), ang = ret
if w<h:
    w,h = h,w
    ang += 90

## (4) Find rotated matrix, do rotation
M = cv2.getRotationMatrix2D((cx,cy), ang, 1.0)
rotated = cv2.warpAffine(threshed, M, (img.shape[1], img.shape[0]))

## (5) find and draw the upper and lower boundary of each lines
hist = cv2.reduce(rotated,1, cv2.REDUCE_AVG).reshape(-1)

th = 2
H,W = img.shape[:2]
uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]

print(len(uppers))
rotated = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)
for y in uppers:
    cv2.line(rotated, (0,y), (W, y), (255,0,0), 1)

for y in lowers:
    cv2.line(rotated, (0,y), (W, y), (0,255,0), 1)

cv2.imwrite("result.png", rotated)

In [ ]:
import cv2
import numpy as np
#import image
image = cv2.imread('/home/topica/Pictures/toan2.png')

#grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

#binary
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)

#dilation
kernel = np.ones((5,5), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=1)

#find contours
im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#sort contours
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
# print(len(sorted_ctrs))
avr_h = 0
a = 0
for i, ctr in enumerate(sorted_ctrs):
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)
    avr_h += h
    a += 1
    
avr_h = avr_h/a
chu = 0
for i, ctr in enumerate(sorted_ctrs):
    x, y, w, h = cv2.boundingRect(ctr)    
    if h >= avr_h/3 and h <= avr_h*3:
        cv2.rectangle(image,(x,y),( x + w, y + h ),(90,0,255),2)
        chu+=1
print(chu)
cv2.imshow('marked areas',image)
cv2.waitKey(0)